In [ ]:
# Merge the two DataFrames (outer join to retain all rows)
import pandas as pd

df1 = pd.read_csv("2017_GSM_parsed.csv", delimiter=",", header="infer")
print(df1.columns)
df2 = pd.read_csv("2023_GSM_parsed.csv", delimiter=",", header="infer")
print(df2.columns)

merged_df = pd.merge(df1, df2, on='id', how='outer', suffixes=('_df1', '_df2'))

# Initialize a final DataFrame to store the cleaned rows
final_rows = []

# Group by the 'id' column to process each group separately
for id, group in merged_df.groupby('id'):
    if len(group) == 1:
        # Case 1: Unique or Identical Row (only one row in group)
        final_rows.append(group.iloc[0])
    else:
        # Case 2: Multiple Rows (Duplicates found)
        combined_row = group.iloc[0].copy()  # Start with the first row
        conflicting_rows = []

        for _, row in group.iterrows():
            for col in row.index:
                if pd.isna(combined_row[col]) and not pd.isna(row[col]):
                    # Fill missing values
                    combined_row[col] = row[col]
                elif combined_row[col] != row[col] and not pd.isna(combined_row[col]) and not pd.isna(row[col]):
                    # Conflict detected
                    conflicting_rows.append(row)

        # If no conflicts, keep the combined row
        if not conflicting_rows:
            final_rows.append(combined_row)
        else:
            # Keep all conflicting rows with unique ids
            counter = 1
            combined_row['id'] = f"{id} ({counter})"
            final_rows.append(combined_row)
            
            print(conflicting_rows)

            for conflict_row in conflicting_rows:
                counter += 1
                conflict_row = conflict_row.copy()
                conflict_row['id'] = f"{id} ({counter})"
                final_rows.append(conflict_row)

# Combine all rows back into a DataFrame
final_df = pd.DataFrame(final_rows)

# Drop intermediate '_df1' and '_df2' suffixes, if created during merge
final_df.columns = [col.split('_')[0] for col in final_df.columns]

# Display final DataFrame
print(final_df)
#duplicates = {'2g_bands':'2g', '3g_bands':'3g', '4g_bands':'4g', 'network_speed', 'gprs', 'edge', 'announced', 'status', 'dimentions':'dimensions', 'sim', 'display_type', 'display_resolution', 'display_size', 'os', 'cpu', 'chipset', 'gpu', 'memory_card':'memory(external)', 'internal_memory':'memory(internal)', 'ram', 'primary_camera', 'secondary_camera', 'loud_speaker', 'audio_jack', 'wlan', 'bluetooth', 'gps', 'nfc', 'radio', 'usb', 'sensors', 'battery', 'colors', 'approx_price_eur', 'img_url'}

Index(['id', 'brand', 'model', 'network_technology', '2g_bands', '3g_bands',
       '4g_bands', 'network_speed', 'gprs', 'edge', 'announced', 'status',
       'dimentions', 'weight_g', 'weight_oz', 'sim', 'display_type',
       'display_resolution', 'display_size', 'os', 'cpu', 'chipset', 'gpu',
       'memory_card', 'internal_memory', 'ram', 'primary_camera',
       'secondary_camera', 'loud_speaker', 'audio_jack', 'wlan', 'bluetooth',
       'gps', 'nfc', 'radio', 'usb', 'sensors', 'battery', 'colors',
       'approx_price_eur', 'img_url'],
      dtype='object')
Index(['unnamed: 0', 'id', '2g', '3g', '4g', '5g', 'announced', 'released',
       'dimension', 'weight', 'build', 'sim', 'body(other)', 'display type',
       'display size', 'resolution', 'display protection', 'display(other)',
       'os', 'chipset', 'cpu', 'gpu', 'memory(external)', 'memory(internal)',
       'main camera', 'main camera(features)', 'main camera(video)',
       'front camera', 'front camera(features)', 'fr